In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib
from simple_dispatch import bidStack
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
colorset = ['#882255','#88CCEE','#332288', '#DDCC77',   '#CC6677','#44AA99', '#117733','#AA4499','#999933']
gds = {}
gds[2019] = pickle.load(open('Data/generator_data_short_WECC_2019.obj', 'rb'))
gds[2020] = pickle.load(open('Data/generator_data_short_WECC_2020.obj', 'rb'))
 

In [ ]:
def plotBidStackMultiColor4(self, df_column, ax=None, xlim=86, save_str=None, show_legend=True, show_fig=True, xlab=False):    
    
    # adapted from Thomas Deetjen's code : https://github.com/tdeetjen/simple_dispatch
    
    bs_df_fuel_color = self.df.copy()
    
    # Add column "fuel_color" to dataframe based on fuel type for each generator
    legend_mapping = {'sub_st':'Subbituminous Coal', 'ng_ct':'Natural Gas Combined Cycle', 
                      'ng_gt':'Natural Gas Combustion Turbine', 'ng_st':'Natural Gas Boiler', 
                      'rc_st':'Refined Coal', 'bit_st':'Bituminous Coal', '_':'Other'}
    c = {'ng': {'cc': colorset[0], 'ct': colorset[1], 'gt': colorset[2], 'st': colorset[3]}, 'sub': {'st': colorset[4]}, 'lig': {'st': str(colorset[5])}, 'bit': {'st': colorset[6]}, 'rc': {'st':  colorset[7]}}

    bs_df_fuel_color['fuel_color'] = colorset[8]
    for c_key in c.keys():
        for p_key in c[c_key].keys():
            bs_df_fuel_color.loc[(bs_df_fuel_color.fuel == c_key) & (bs_df_fuel_color.prime_mover == p_key), 'fuel_color'] = c[c_key][p_key]
        empty_color = '#dd1c77'
        color_2 = bs_df_fuel_color.fuel_color.replace('', empty_color)
        
    # prepare y depending on whether plotting cost or emissions
    y_data_e = self.df.gen_cost * 0
    if df_column == 'gen_cost':
        y_lab = 'Generation Cost [$/MWh]'
        y_data = self.df[df_column] - (self.df.co2_cost + self.df.so2_cost + self.df.nox_cost) #cost excluding emissions taxes
    if df_column == 'co2':
        y_lab = 'CO$_2$ Emissions [kg/MWh]'
        y_data = self.df[df_column + str(self.time)]

    # plot
    if ax is None:
        f, ax = matplotlib.pyplot.subplots(figsize=(4, 4))
    # Bar chart: width of each is the capacity. Shift position by one indice so that the bar lines up to the left of its capacity. E.g. if the first one has capacity 10, the bar goes from 0 to 10 and the following bar starts from 10. 
    ax.bar((self.df['mw' + str(self.time)].cumsum()[:-1])/1000, height=y_data.values[1:], width=(self.df['mw' + str(self.time)].values[1:])/1000, color=color_2.values[1:], align='edge', alpha=.8)
    if show_legend:
        color_legend = []

        for c in bs_df_fuel_color.fuel_color.unique():
            color_legend.append(matplotlib.patches.Patch(color=c, label=legend_mapping[bs_df_fuel_color.fuel[bs_df_fuel_color.fuel_color==c].iloc[0] + '_' + bs_df_fuel_color.prime_mover[bs_df_fuel_color.fuel_color==c].iloc[0]]))
        ax.legend(handles=color_legend, loc='upper left', ncol=1, fontsize=10.3)
    
    xmin = 40
    ax.set_xlim(xmin, xlim)
    if df_column == 'gen_cost':
        ax.set_ylim(0, 90)
        ax.set_yticks((0, 15, 30, 45, 60, 75, 90, 105))
        ax.set_yticklabels((0, 15, 30, 45, 60, 75, 90, 105), fontsize=18)
    elif df_column == 'co2':
        ax.set_ylim(0, 1400)
        ax.set_yticks((250, 500, 750, 1000, 1250))
        ax.set_yticklabels((250, 500, 750, 1000, 1250), fontsize=18)
    if xlab:
        ax.set_xlabel('Fossil Generation Capacity [GW]', fontsize=20)
        ax.set_xticklabels(np.arange(xmin, xlim+1, 2), fontsize=20)
    else:
        ax.set_xticklabels([])
    
    ax.set_xticks(np.arange(xmin, xlim+1, 2))
    ax.set_axisbelow(True)        
    ax.grid()

    if save_str is not None:
        plt.savefig(save_str, bbox_inches='tight')
    if show_fig:
        plt.show()
        
    
    return ax

In [ ]:
xlim = 50

f, axes = plt.subplots(5, 1, figsize=(8,12))
bs = bidStack(gds[2020], co2_dol_per_kg=0, time=2, dropNucHydroGeo=True, include_min_output=False, mdt_weight=0.5, include_easiur=False) 

axes[0] = plotBidStackMultiColor4(bs, 'co2', ax=axes[0], show_legend=True, show_fig=False, xlim=xlim, xlab=False)
axes[1] = plotBidStackMultiColor4(bs, 'co2', ax=axes[1], show_legend=False, show_fig=False, xlim=xlim, xlab=False)
axes[2] = plotBidStackMultiColor4(bs, 'co2', ax=axes[2], show_legend=False, show_fig=False, xlim=xlim, xlab=False)
axes[3] = plotBidStackMultiColor4(bs, 'co2', ax=axes[3], show_legend=False, show_fig=False, xlim=xlim, xlab=False)
axes[4] = plotBidStackMultiColor4(bs, 'co2', ax=axes[4], show_legend=False, show_fig=False, xlim=xlim, xlab=True)

f.text(-.035, 0.5, 'CO$_2$ Emissions [kg/MWh]', va='center', rotation='vertical', fontsize=20)
#group 1, y=5.44, 3.86, 2.68, 1.47
# Demand, MEF
# 49190.28272, 882.1342434
# 48475.40726, 388.8263059
# 46641.93642, 370.0359868
# 44675.30245, 359.3285499
# 42166.05406, 627.0249265
#5.42, 4.79, 4.82, 4.58, 4.6, 3.55, 3.57, 3.4, 3.41, 2.39, 2.41, 2.2, 2.22, 1.18, 1.2, 1, -.01

plt.annotate('',
            xy=(.92, 5.40), xycoords='axes fraction',
            xytext=(0.92, 4.79), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.923, 4.813), xycoords='axes fraction',
            xytext=(0.846, 4.587), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.848, 4.61), xycoords='axes fraction',
            xytext=(0.848, 3.912), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.848, 3.843), xycoords='axes fraction',
            xytext=(0.848, 3.59), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.851, 3.605), xycoords='axes fraction',
            xytext=(0.66, 3.395), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.664, 3.41), xycoords='axes fraction',
            xytext=(0.664, 2.69), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.664, 2.6392), xycoords='axes fraction',
            xytext=(0.664, 2.39), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.668, 2.41), xycoords='axes fraction',
            xytext=(0.463, 2.192), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.468, 2.215), xycoords='axes fraction',
            xytext=(0.468, 1.482), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.468, 1.43), xycoords='axes fraction',
            xytext=(0.468, 1.19), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.472, 1.2), xycoords='axes fraction',
            xytext=(0.213, 1), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217,1.01), xycoords='axes fraction',
            xytext=(0.217, .482), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217,.43), xycoords='axes fraction',
            xytext=(0.217, -.01), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')


# #group 2: 5.44, 3.89, 2.73, 1.47, .43
# # 49189.0694, 882.1342434
# # 48517.50048, 388.8263059
# # 46804.29771, 457.5367826
# # 45965.55552, 375.0393161
# # 42174.65084, 627.0249265


plt.annotate('',
            xy=(.92, 5.40), xycoords='axes fraction',
            xytext=(0.92, 4.79), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.923, 4.813), xycoords='axes fraction',
            xytext=(0.848, 4.587), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.85, 4.61), xycoords='axes fraction',
            xytext=(0.85, 3.912), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.85, 3.842), xycoords='axes fraction',
            xytext=(0.85, 3.59), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.85, 3.605), xycoords='axes fraction',
            xytext=(0.675, 3.395), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.679, 3.41), xycoords='axes fraction',
            xytext=(0.679, 2.761), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.68, 2.7), xycoords='axes fraction',
            xytext=(0.68, 2.39), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')

plt.annotate('',
            xy=(.683, 2.412), xycoords='axes fraction',
            xytext=(0.593, 2.188), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.596, 2.215), xycoords='axes fraction',
            xytext=(0.596, 1.48), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.596, 1.432), xycoords='axes fraction',
            xytext=(0.596, 1.185), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.60, 1.202), xycoords='axes fraction',
            xytext=(0.214, 1), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217, 1), xycoords='axes fraction',
            xytext=(0.217, .482), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217, .415), xycoords='axes fraction',
            xytext=(0.217, -.01), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')

# #group 3, 5.44, 3.89, 2.73, 1.52, .43
# # 49190.0617, 882.1342434
# # 48581.9323, 388.8263059
# # 46831.80811, 457.5367826
# # 46861.55997, 457.5367826
# # 42183.99544, 627.0249265


plt.annotate('',
            xy=(.92, 5.40), xycoords='axes fraction',
            xytext=(0.92, 4.79), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.923, 4.813), xycoords='axes fraction',
            xytext=(0.848, 4.587), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.85, 4.61), xycoords='axes fraction',
            xytext=(0.85, 3.91), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.85, 3.839), xycoords='axes fraction',
            xytext=(0.85, 3.59), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.854, 3.605), xycoords='axes fraction',
            xytext=(0.68, 3.395), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.683, 3.413), xycoords='axes fraction',
            xytext=(0.683, 2.733), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.683, 2.71), xycoords='axes fraction',
            xytext=(0.683, 2.39), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')

plt.annotate('',
            xy=(.683, 2.412), xycoords='axes fraction',
            xytext=(0.686, 2.188), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.686, 2.215), xycoords='axes fraction',
            xytext=(0.686, 1.961), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.686, 1.895), xycoords='axes fraction',
            xytext=(0.686, 1.185), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.69, 1.202), xycoords='axes fraction',
            xytext=(0.214, 1), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217,1.017), xycoords='axes fraction',
            xytext=(0.217, .482), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217,.415), xycoords='axes fraction',
            xytext=(0.217, -.01), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1   ),
            horizontalalignment='right', verticalalignment='top')


# #group 4, 5.44, 3.89, 2.73, 1.92, .43
# #49189.67063, 882.1342434
# #48683.86395,421.5340827
# #46861.43215,457.5367826
# #46888.52922, 1016.31584
# #42193.3007, 627.0249265


plt.annotate('',
            xy=(.92, 5.40), xycoords='axes fraction',
            xytext=(0.92, 4.79), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.923, 4.815), xycoords='axes fraction',
            xytext=(0.865, 4.587), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.868, 4.61), xycoords='axes fraction',
            xytext=(0.868, 3.91), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.868, 3.884), xycoords='axes fraction',
            xytext=(0.868, 3.59), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.871, 3.607), xycoords='axes fraction',
            xytext=(0.684, 3.395), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.686, 3.41), xycoords='axes fraction',
            xytext=(0.686, 2.73), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.686, 2.72), xycoords='axes fraction',
            xytext=(0.686, 2.39), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.686, 2.42), xycoords='axes fraction',
            xytext=(0.688, 2.18), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.688, 2.215), xycoords='axes fraction',
            xytext=(0.688, 1.961), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.688, 1.895), xycoords='axes fraction',
            xytext=(0.688, 1.185), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.690, 1.2), xycoords='axes fraction',
            xytext=(0.214, 1), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217, 1.015), xycoords='axes fraction',
            xytext=(0.217, .482), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.217, .415), xycoords='axes fraction',
            xytext=(0.217, -.01), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')

#group 5: 5.44, 3.91, 3.13, 1.92, .43
# 49191.57875, 882.1342434
# 48759.23212, 421.5340827
# 46903.50202, 1016.31584
# 46888.81601, 1016.31584
# 42213.37941, 627.0249265

plt.annotate('',
            xy=(.919, 5.4), xycoords='axes fraction',
            xytext=(0.919, 4.79), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(0.922, 4.82), xycoords='axes fraction',
            xytext=(0.874, 4.585), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(0.876, 4.61), xycoords='axes fraction',
            xytext=(0.876, 3.94), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(0.876, 3.858), xycoords='axes fraction',
            xytext=(0.876, 3.585), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.88, 3.605), xycoords='axes fraction',
            xytext=(0.686, 3.395), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.69, 3.415), xycoords='axes fraction',
            xytext=(0.69, 3.168), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')

plt.annotate('',
            xy=(.69, 3.083), xycoords='axes fraction',
            xytext=(0.688, 2.185), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')

plt.annotate('',
            xy=(.688, 2.22), xycoords='axes fraction',
            xytext=(0.688, 1.961), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.688, 1.895), xycoords='axes fraction',
            xytext=(0.688, 1.185), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.693, 1.20), xycoords='axes fraction',
            xytext=(0.2213, 1), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')

plt.annotate('',
            xy=(.2213, 1.015), xycoords='axes fraction',
            xytext=(0.2213, .49), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')
plt.annotate('',
            xy=(.2213, .415), xycoords='axes fraction',
            xytext=(0.2213, -.012), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', arrowstyle='-', linewidth=1  ),
            horizontalalignment='right', verticalalignment='top')

#1
axes[0].plot(49.19028272, 882.1342434, 'o', color='1', markersize=10, markeredgecolor='k')
axes[1].plot(48.47540726, 388.8263059, 'o', color='1', markersize=10, markeredgecolor='k',label='Group 1')
axes[2].plot(46.64193642, 370.0359868, 'o', color='1', markersize=10, markeredgecolor='k')
axes[3].plot(44.675, 359.3285499, 'o', color='1',  markersize=10, label='Group 1', markeredgecolor='k')
axes[4].plot(42.16605406, 627.0249265, 'o', color='1', markersize=10, markeredgecolor='k',label='Group 1')

#2
axes[0].plot(49.189, 882.1342434, 'P', color='1', markersize=10, markeredgecolor='k')
axes[1].plot(48.5175, 388.8263059, 'P', color='1', markersize=10, markeredgecolor='k',label='Group 2')
axes[2].plot(46.8042, 457.5367826, 'P', color='1', markersize=10, markeredgecolor='k')
axes[3].plot(45.9655, 375.0393161, 'P', color='1', markersize=10, markeredgecolor = 'k', label='Group 2')
axes[4].plot(42.1746, 627.0249265, 'P', color='1', markersize=10, markeredgecolor='k',label='Group 2')


#3
axes[0].plot(49.1890694, 882.1342434, '*', color='1', markersize=10, markeredgecolor='k')
axes[1].plot(48.51750048, 388.8263059, '*', color='1', markersize=10, markeredgecolor='k',label='Group 3')
axes[2].plot(46.80429771, 457.5367826, '*', color='1', markersize=10, markeredgecolor='k')
axes[3].plot(45.96555552, 375.0393161, '*', color='1', markersize=10, label='Group 3', markeredgecolor='k')
axes[4].plot(42.17, 627.0249265, '*', color='1', markersize=10, markeredgecolor='k',label='Group 3')


#4
axes[0].plot(49.18967063, 882.1342434, 'X', color='1', markersize=10, markeredgecolor='k')
axes[1].plot(48.68386395,421.5340827, 'X', color='1', markersize=10, markeredgecolor='k',label='Group 4')
axes[2].plot(46.86143215, 457.5367826, 'X', color='1',markersize=10, markeredgecolor='k')
axes[3].plot(46.88852922, 1016.31584, 'X', color='1', label='Group 4', markersize=10, markeredgecolor='k')
axes[4].plot(42.1933007, 627.0249265, 'X', color='1', markersize=10, markeredgecolor='k',label='Group 4')

#5
axes[0].plot(49.19, 882.1342434, 'd', color='1', markersize=9, markeredgecolor='k')
axes[1].plot(48.75923212, 421.5340827, 'd', color='1', markersize=9, markeredgecolor='k',label='Group 5')
axes[2].plot(46.90350202, 1016.31584, 'd', color='1', markersize=9, markeredgecolor='k')
axes[3].plot(46.88, 1016.31584, 'd', color='1', label='Group 5', markersize=9, markeredgecolor='k')
axes[4].plot(42.213, 627.0249265, 'd', color='1', markersize=9, markeredgecolor='k',label='Group 5')


plt.annotate('',
            xy=(1.1, 1.8), xycoords='axes fraction',
            xytext=(1.1, 4), textcoords='axes fraction', color=colorset[0], fontsize =18,
            arrowprops=dict(linewidth=5, color=colorset[0] ),
            horizontalalignment='center', verticalalignment='top')

plt.text(1.06, -.6,"Time",  transform=axes[0].transAxes, color=colorset[0], fontsize =20)

plt.text(.63, .77,"Hour 18",  transform=axes[0].transAxes, fontsize =18)
plt.text(.63, -.43,"Hour 19",  transform=axes[0].transAxes, fontsize =18)
plt.text(.83, -1.63,"Hour 20",  transform=axes[0].transAxes, fontsize =18)
plt.text(.83, -2.83,"Hour 21",  transform=axes[0].transAxes, fontsize =18)
plt.text(.83, -4.03,"Hour 22",  transform=axes[0].transAxes, fontsize =18)


axes[1].legend( loc='upper left', ncol=1, fontsize=13.5)
plt.savefig('Merit_Order/7a_mef_vis.pdf', bbox_inches='tight')
plt.show()